In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['IMDB Dataset.csv']


In [2]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re

import warnings
warnings.filterwarnings('ignore')


In [3]:
imdb_data=pd.read_csv('../input/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [4]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [5]:
imdb_data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [6]:
#IMDB dataset
reviews=np.array(imdb_data['review'])
sentiments=np.array(imdb_data['sentiment'])
 #Split the dataset
train_reviews=reviews[:40000]
train_sentiments=sentiments[:40000]
test_reviews=reviews[40000:]
test_sentiments=sentiments[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [7]:
#Text preprocessing or text normalization
#Noise removal
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

def strip_html(review):
    soup=BeautifulSoup(review,"html.parser")
    return soup.get_text

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
        
def denoise_text(text):
    trxt=strip_html(text)
    text=remove_between_square_brackets(text)
    return text

imdb_data['review']=imdb_data['review'].apply(denoise_text)
#remove accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
imdb_data['review']=imdb_data['review'].apply(remove_accented_chars)

CONTRACTION_MAP = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 
#Expand contractions
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text
imdb_data['review']=imdb_data['review'].apply(expand_contractions)

#remove special characters
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

#Text stemming
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

#Remove stopwords
stop=set(stopwords.words('english'))
print(stop)

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)
imdb_data.review[0]

{"won't", 'against', 'couldn', 'me', 'shouldn', 'and', "mightn't", 'should', 'but', 'again', 'hadn', "you'd", 'needn', 'were', 't', 'own', 'this', 'himself', 'there', 'or', 'before', 'no', 'once', 'the', 'because', 'then', "should've", 'them', 'is', 'yourself', 'been', 'ain', "weren't", 'at', 'who', 'ourselves', 'out', 'mustn', 's', 'yourselves', 'under', 'my', 'will', 'ma', 'was', 'those', 'just', "hadn't", 'when', "needn't", 'any', 'having', "isn't", 'hers', 'd', 'itself', 'an', "it's", 'more', 'with', 'such', 'both', 'than', 'mightn', 'll', 'very', 'between', 'shan', "wouldn't", 'did', 'won', 'they', 'about', 'all', 'down', "doesn't", 'its', 've', 'other', 'him', 'didn', 'herself', 'in', 'now', 'doing', 'into', "couldn't", 'through', 'aren', 'further', 'does', "hasn't", 'isn', 'yours', 'until', 'on', 'during', 'which', 'wasn', 'm', 'a', 'nor', 'so', 'for', "you'll", 'above', 'it', 'where', 'their', 'after', 'most', "that'll", 're', 'have', 'why', 'haven', 'he', 'too', 'weren', "she'

'one review ha mention watch 1 Oz episod hook right thi exactli happen mebr br first thing struck Oz wa brutal unflinch scene violenc set right word GO trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordbr br call OZ nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda Em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awaybr br would say main appeal show due fact goe show would dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doe mess around first episod ever saw struck nasti wa surreal could say wa readi watch develop tast Oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch Oz ma

In [8]:
#Split the  dataset
norm_train_reviews=imdb_data.review[:40000]
norm_train_reviews

0        one review ha mention watch 1 Oz episod hook r...
1        wonder littl product br br film techniqu veri ...
2        thought thi wa wonder way spend time hot summe...
3        basic famili littl boy jake think zombi hi clo...
4        petter mattei love time money visual stun film...
5        probabl alltim favorit movi stori selfless sac...
6        sure would like see resurrect date seahunt ser...
7        thi show wa amaz fresh innov idea 70 first air...
8        encourag posit comment thi film wa look forwar...
9        like origin gut wrench laughter like thi movi ...
10       phil alien one quirki film humour base around ...
11       saw thi movi wa 12 came recal scariest scene w...
12       im big fan boll work mani enjoy hi movi postal...
13       cast play shakespearebr br shakespear lostbr b...
14       thi fantast movi three prison becom famou one ...
15       kind drawn erot scene onli realiz thi wa one a...
16       film simpli remad thi one bad film fail captur.

In [9]:
#normalization of test reviews
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews

40000    first want say lean liber polit scale found mo...
40001    wa excit see sitcom would hope repres indian c...
40002    look cover read stuff entir differ type movi c...
40003    like mani count appear denni hopper make thi c...
40004    thi movi wa tv day enjoy first georg jungl wa ...
40005    hickori dickori dock wa good poirot mysteri co...
40006    one crazi summer funniest craziest necessarili...
40007    lowbudget schlockmeist herschel gordon lewi re...
40008    thi wa absolut tragic pile cinema ever born wi...
40009    never understood appeal thi show act poor debr...
40010    Mr bug goe town wa last major achiev fleischer...
40011    thi one two three favorit stoog short undoubte...
40012    thi suppos base wilki collin _the_woman_in_whi...
40013    british imperialist movi like four feather cha...
40014    jame cagney racket polit ward heeler get becom...
40015    love thi film swoon Ed wood Jr fan prefer appr...
40016    alreadi seen origin jack frost never thought j.

In [10]:
#Bags of words of model
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,2))
cv_train_reviews=cv.fit_transform(norm_train_reviews)
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (40000, 1836705)
BOW_cv_test: (10000, 1836705)


In [11]:
#TfidfVectorizer model
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,2),sublinear_tf=True)
tv_train_reviews=tv.fit_transform(norm_train_reviews)
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 1836705)
Tfidf_test: (10000, 1836705)


In [12]:
from sklearn.preprocessing import LabelBinarizer
lb=LabelBinarizer()
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

#split the sentiment data
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]

#Supervised learning models
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix

lr=LogisticRegression(penalty='l2',max_iter=100,C=1,random_state=0)

svm=SGDClassifier(loss='hinge',n_iter=100,random_state=0) # SGD(Stochastic Gradient Descent)
#Loggistic regression for BOW model
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)
lr_bow_predict=lr.predict(cv_test_reviews)
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#From the confusion matrix,as we can predict that out of 4993 reviews, 2890 reviews are correctly predicted as positive reviews,
#and out of 5007 reviews,3889 reviews are correctly predicted as negative reviews.

(50000, 1)
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
              precision    recall  f1-score   support

    Positive       0.65      0.78      0.71      4993
    Negative       0.72      0.58      0.64      5007

   micro avg       0.68      0.68      0.68     10000
   macro avg       0.69      0.68      0.67     10000
weighted avg       0.69      0.68      0.67     10000

[[2890 2117]
 [1104 3889]]


In [13]:
#Logistic Regression for Tfidf model
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
lr_tfidf_predict=lr.predict(tv_test_reviews)
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['positive','negative'])
print(lr_tfidf_report)
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict)
print(cm_tfidf)
#From the confusion matrix,as we can predict that out of 4993 reviews, 3432 reviews are correctly predicted as positive reviews,
#and out of 5007 reviews,3435 reviews are correctly predicted as negative reviews.

              precision    recall  f1-score   support

    positive       0.69      0.69      0.69      4993
    negative       0.69      0.69      0.69      5007

   micro avg       0.69      0.69      0.69     10000
   macro avg       0.69      0.69      0.69     10000
weighted avg       0.69      0.69      0.69     10000

[[3432 1561]
 [1572 3435]]


In [14]:
#SVM for BOW model
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
svm_bow_pred=svm.predict(cv_test_reviews)
svm_bow_report=classification_report(test_sentiments,svm_bow_pred,target_names=['Positive','Negative'])
cm_bow=confusion_matrix(test_sentiments,svm_bow_pred)
print(svm_bow_report,cm_bow)
#From the confusion matrix,as we can predict that out of 4993 reviews,3631 reviews are correctly predicted as positive reviews,
#and out of 5007 reviews,3090 reviews are correctly predicted as negative reviews.

              precision    recall  f1-score   support

    Positive       0.65      0.73      0.69      4993
    Negative       0.69      0.62      0.65      5007

   micro avg       0.67      0.67      0.67     10000
   macro avg       0.67      0.67      0.67     10000
weighted avg       0.67      0.67      0.67     10000
 [[3631 1362]
 [1917 3090]]


In [15]:
#SVM for Tfidf model
svm_tf=svm.fit(tv_train_reviews,train_sentiments)
svm_tf_pred=svm.predict(tv_test_reviews)
svm_tf_report=classification_report(test_sentiments,svm_tf_pred,target_names=['Positive','Negative'])
cm_tf=confusion_matrix(test_sentiments,svm_tf_pred)
print(svm_tf_report, cm_tf)
#From the confusion matrix,as we can predict that out of 4993 reviews, 4993 reviews are correctly predicted as positive reviews,
#and out of 5007 reviews,0 reviews are correctly predicted as negative reviews.

              precision    recall  f1-score   support

    Positive       0.50      1.00      0.67      4993
    Negative       0.00      0.00      0.00      5007

   micro avg       0.50      0.50      0.50     10000
   macro avg       0.25      0.50      0.33     10000
weighted avg       0.25      0.50      0.33     10000
 [[4993    0]
 [5007    0]]
